# Fifa Position Predictor

- Ali Elmancy 201901993
- Ahmed Abdou 202005605
- Ahmed Terchoun 201802794

## Abstract
In this project, we investigated the relationship between player attributes and positions in FIFA, a popular video game and football simulator. We used the FIFA 2015-2023 complete player dataset, which is available on [Kaggle/FIFA](https://www.kaggle.com/datasets/stefanoleone992/fifa-23-complete-player-dataset?resource=download&select=male_players+%28legacy%29.csv). This dataset includes data for male teams, coaches, and two datasets for male players, one of which is 5.5GB and the other is 90MB. For our purposes, we used the 90MB dataset, which contains over 160,000 records for all players since FIFA 15, each with 110 different features. As for the first phase of our project, we focused on predicting the single most suitable position for a player based on their attributes. In the second phase, we will focus on predicting and estimating the probability of a player playing in each position.

### 1. Data Cleaning

At this stage of the project we began by dividing the attributes under two categories, main and detailed, according to our knowledge of the game. The main attributes represent player stats present on his card while detailed ones are internal and usually reflect the main stats but contain extra attributes. We did neglect some other columns that are irrelevant to the task such as the player club and shirt number in order to reduce the dimensionality and provide the model accurate data. After doing so we ended up with a total of 42 columns. 

Furthermore, we dropped all goalkeepers as their stats are totally different from other positions and they can be easily distinguished. Also, the column mentality_composure was dropped due to having almost 18k nan values.

After that, we went to process our target column which is player_positions, it included a list of positions in which a player can play in. However, we created our own encoding class so that we can map player positions into one of seven main categories [ST, LW, RW, CM, LB, RB, CB]. We chose the player's first position to be mapped as it is the position at which the player performs the best. Then we renamed this column to positions and added it to the end of the dataframe.

After this step we saved a copy of this dataframe called cleaned_df which is later used for EDA. Moving forward we had two categorical columns, preferred_foot and work_rate, that had text values, so we used one-hot encoding technique to transform these columns so that they can be used for neural networks. The main reason behind choosing one-hot encoding over other encoding techniques is that it does not assign any priorty to the values unlike label encoding. That is why it worked best for our dataset.

Moreover, we used our encoding function from the coder class to encode the players' positions to numerical values from 0 to 6, and saved this version of the dataframe as encoded_df which is going to be used later for PCA.

Finally, we proceeded by normalizing all the continuous values columns in order to prepare the data for training. We saved this version of the dataframe as normalized_df. The final shape of the dataframe after dropping the unnecessary columns and encoding the categorical ones is 143613 rows and 50 columns. 

### 2. EDA

Starting out with the players' physical attributes, it is clear that there exists a strong psoitive correlation between the height and weight of a player and the positions they play in, visualized usign a scatter plot. This is intuitive, as taller and heavier players are more likely to play either as defenders or strikers than a midfielder. This plays a key role in intially filtering out the positions that a player is likely to play in.

When it came to the players preferred foot, using barplots, we found that the majority of players are rigth footed which is expected as the majority of the world's population is right handed. However, the majority of LB players were found to be left footed while almost all RB are right footed! This features was found pretty indicative of wether a player is a LB or not liekly to be.

Moving into players' weak foot stars & skill moves stars, we used a heatmap to visualzie the percentages of players for each star levels in each position. We found that the majority of players have 3 stars in both features, which is expected as the majority of players are average. However, players playing on the upper side of the pitch where foudn to have better ratings for both their weak foots & skill moves compared to those playing on the bottom part of the pitch.

Now for the 6 main attributes displayed on each player's fifa card.
We used boxplots and distribution plots to visualize each attribute against each position. Here were our findings:

- **Pace**: CBs were significantly slower than the rest, while attacking wingers were the fastest of them all. Strickers and defending wingers had relatively similar pace ratings, while midfielders were inbetween the runner ups to CB as one of the slower positions.

- **Shooting**: Strickers were the best shooters, followed by attacking wingers then the midfeilders. Defending wingers were significantly worse in shooting than those positions, but the center backs were out of competition with the rest of the positions.

- **Passing**: Similar to shooting, center defenders were lacking by miles compared to the rest of the positions. This time however, the midfielders were the best passers follwed by the attacking wingers. Strickers this time, did not have the advantage over the defending wingers in passing comming the second worst place when it came to passing.

- **Dribbling**: Attacking wingers were the best dribblers, followed by the midfielders. Strickers  and defending wingers were a bit behind compared to the top 3 positions but having a similar rating. Center backs were the worst at this catgeory again.

- **Defending**: This was the only attribute where center backs excelled at by a margin, followed by the defending wingers then midfeilders who had a wide range of skill ratings (some are CAMs and others are CDMs). Attacking wingers and strikers were the worst at defending but stricker were alot behind on the spectrum.

- **Physical**: Center backs were the most physical players, followed by the defending wingers. Strickers and midfielders were very close in their physical ratings, while attacking wingers were the least physical players.


Each of these attributes had a similar set of hidden attributes that fifa has not displayed on the players' cards. but contributed to the overall rating of the player. We used seaborn's magical pairplots to visualize the correlation between each of the main attributes and their hidden counterparts. Here were our findings:

*Here we take in consideration the highest correlation between their corresponding main attributes*


- **Shooting** & **Passing**: The hidden attributes that contributed to the shooting rating had these corresponding correlations -> 
    - **attacking_finishing**: 0.99
    - **attacking_volleys**: 0.99
    - **attacking_heading_accuracy**: -0.90
    - **attacking_crossing**: 0.80
As seen all of these attributes do not contirbute much to the dataset when it comes to predicting the players' position as the main attributes summarize them all.

- **Dribbling** & **Passing**: The hidden attributes that contributed to the dribbling rating had these corresponding correlations-> 
    - **skill_dribbling**: 0.99
    - **skill_ball_control**: 0.98
    - **skill_curve**: 0.99
    - **skill_fk_accuracy**: 0.95
    - **movement_vision**: 0.98

- **Dribbling** & **Physic**: The hidden attributes that contributed to the dribbling rating had these corresponding correlations-> 
    - **movement_agility**: 0.98
    - **movement_reactions**: 0.29 *Low correlation*
    - **movement_balance**: 0.98
    - **movement_sprint_speed**: 0.99
    - **movement_acceleration**: 0.99

- **Defending**: The hidden attributes that contributed to the defending rating had these corresponding correlations-> 
    - **defending_marking_awarness**: 0.999
    - **defending_standing_tackle**: 0.999
    - **defending_sliding_tackle**: 0.997

- **Physi**: The hidden attributes that contributed to the physical rating had these corresponding correlations-> 
    - **power_shot_power**: 0.99
    - **power_jumping**: 0.94
    - **power_strength**: 0.91
    - **power_long_shots**: -0.80
    - **power_stamina**: 0.2 *Low correlation*

- **All**: The hidden attributes that contributed to most of the main attributes had these corresponding correlations-> 
    - **mentality_aggression**: 0.89
    - **mentality_interceptions**: 0.92
    - **mentality_positioning**: 0.93
    - **mentality_vision**: 0.91
    - **mentality_penalties**: 0.98

Lastly, we wanted to see if the workrate attribute had any effect on the players' positions. the workrate are set of indicators for the player's attacking and defensive workrates. We used a heatmap to visualize the percentage of players for each workrate combination in each position. We found that the majority of players have a medium/medium workrate combination, The visualization was a bit shocking, we thoughtwork rate would have a much bigger effect on the players' positions but it seems like it has a very a mutual effect. Only certain values for the workrate actually differentiated between attacking and defensive players. So overall, it provides somewhat of an indication.

    

### 3. Feature Selection

In this part of the project we tried to filter out the unnecessary features and pick up the ones that contribute the most to the output of the classifier.

We began by dividing the features of our dataset into seven categories:

- MAIN_FIFA_CARD_FEAURES
- ATTACKING_FEATURES
- SKILL_FEATURES
- MOVE_FEATURES
- POWER_FEATURES
- MENTAL_FEATURES
- DEFENDING_FEATURES

Based on the results we got from EDA, we combined weight and height into one column by taking the sum and dividing by 2. We then iterated through the main_features and every other feature to capture the correlation among them. Any feature that had a correlation greater than |0.8| with one of the main features was added to a list of columns to be dropped. The reasoning behind this step is that most of the internal features appeared to have high correlation with the main stats of the player from the done EDA, and removing them would reduce the redundancy of information.

After removing these features, we wanted to figure out which columns have weak correlation with our target. Therefore, we iterated over all the columns and dropped any column that has a correlation less than or equal to |0.1| with the position column. By this step, we concluded the manual feature selection.

As we wanted to further experiment with feature selection, we used PCA to reduce the dimensionality of the dataset while keeping most of the information present. We tried different ranges of components and chose the best one, which is 12 as it keeps 80% of the dataset variance. With the new components, the difference in accuracy while testing with initial models was minimal, so we might continue working with them. 


### 4. Model
At this stage we tried couple of initial models to a get a baseline performance for our final model. The initial model we decided to try was RandomForestClassifier since our task is classification. The training accuracy of the model on the final data was 0.84 and testing accuracy was 0.84 as well. However, when we used the same classifier on the pca data we got a training accuracy of 0.8 and testing accuracy of 0.796. From the previous results we can see that the performance is not that different considering that when using pca the number of features is less than 45% of the final data. 